In [1]:
import ipywidgets as widgets
from ipywidgets import Button, Layout, Output, Select, HBox,VBox
from lib.utils import HBoxAlign
from IPython.display import display
import os
from pathlib import Path
import sys
from glob import glob

sys.path.append("../../")
import lib.engine as engine


In [2]:
output_init = Output(layout={'border': '0px solid transparent'})
def init_tool(DATASET_PATH, OUTPUT_PATH):
    with output_init:
        display(widgets.HBox(children=[widgets.HTML("<h1>Loading... <i class='fas fa-compact-disc fa-spin'></i></h1>",)],layout=Layout(height='auto', width='auto', justify_content='center')))
    
    sub_classes = ['big','medium','small']

    labels=[ 
        'color',
        'black',
        'white',
        'mix'
    ] 

    # Folder generation
    for l in labels:
        for c in sub_classes: 
            Path(f'{OUTPUT_PATH}/{l}/{c}').mkdir(exist_ok=True, parents=True)

    Path(f'{OUTPUT_PATH}/Discard').mkdir(exist_ok=True, parents=True)
    Path(f'{OUTPUT_PATH}/Skip').mkdir(exist_ok=True, parents=True)

    # Listing files
    toLabel = [(r, d, files) for r, d, files in os.walk(DATASET_PATH)][0][-1]
    labeled = list(filter(lambda x: len(x[1])==0, [(r, d, files) for r, d, files in os.walk(OUTPUT_PATH)]))
    
    to_label_imgs=[]

    for img in toLabel:
        toLabel=True
        for class_folders in labeled:
            if img in class_folders[-1]:
                toLabel=False
                break
        if toLabel:
            to_label_imgs.append(img)
                
    to_label_imgs=[os.path.join(DATASET_PATH,x) for x in to_label_imgs]            
    LD = engine.labeling_display(to_label_imgs, labels, sub_classes,  OUTPUT_PATH)

    with output_init:
        display(LD.controls, clear=True)
    
    display(LD.output_images)
    display(LD.output_progress)

In [3]:
users=['rama',]
output_ds=['dogsLabeled']
def parse_options():    
    DATASET_PATH=f'{sub_ds_options.value}'
    OUTPUT_PATH=f'./labeled_data_{output_ds_options.value}_{user_options.value}'
    init_tool(DATASET_PATH, OUTPUT_PATH)

btn_go = Button(description='Continue', button_style='success', icon='fa-check', layout=Layout(height='auto', width='auto'))
btn_go.on_click(lambda _: parse_options())

user_options=Select(
    options=users,
    value=users[0],
    description='User:',
    disabled=False,
)
ds_options_list = glob('./data/*')
sub_ds_options = Select(
    options=ds_options_list,
    value=ds_options_list[0],
    description='Source DS:',
    disabled=False,
)

output_ds_options=Select(
    options=output_ds,
    value=output_ds[0],
    description='Output Dataset:',
    disabled=False,
)

initbox = HBox(children=[user_options,sub_ds_options,output_ds_options], layout=Layout(height='auto', width='auto', justify_content='center')) 
init_grid = VBox(children=[
    initbox,
    HBoxAlign([btn_go], justify='center')
])

display(output_init)
with output_init:
    display(init_grid)

Output(layout=Layout(border='1px solid transparent'))

Output(layout=Layout(border='0px solid transparent'))

Output(layout=Layout(border='0px solid transparent'))